# Tarea 1: Reconocimiento de Patrones en Minería de Datos


## Parte I: Análisis de rendimiento de distintos algoritmos de clustering

Ideas interesantes:
* Graficar siempre los clusters (colores) obtenidos.
* Utilizar un contraste de hipótesis para deteminar la mejor combinación de hiperparámetros.
* 8


# Benchmark Analítico





| Modelo | Param Grid | # Modelos Estimables (multiplicados por CV) | Avg. Precision | Avg. Recall | Avg. F1-Score 
|--------|------------|----------------------|
| K-Means (Random Init.) | n_clusters |  | 
| Support Vector Classifiers | `C`, `gamma` | 320 | 
| Decision Tree Classifiers | `criterion`, `max_depth`, `min_samples_split`, `max_features` | 50.000 |
| AdaBoost Classifier | `n_estimators`, `learning_rate` | 500 | 
| Gradient Boosting Classifier | `loss`, `learning_rate`, `n_estimators`, `subsample` | 2.500 | 
| Random Forest Classifier | `criterion`, `max_depth`, `min_samples_split`, `max_features`, `n_estimators` | 60.000| 
| Naive Bayes Classifier | `alpha` | 18 | 
| Knn | `n_clusters` | 500 | 

* **Nota:** Se reportan los estadísticos medios entre las clases


Modelo|Average Precisión|Average Recall| Average F1-Score
------|---------------|----------|------
Árbol de Clasificación | 0.54 | 0.52 | 0.53
Random Forest | 0.61 | 0.63 | 0.56
Naive Bayes | 0.60 | 0.55 | 0.56
Logit | 0.69 | 0.64 | 0.65
SVC | 0.41 | 0.63 | 0.50



In [45]:
# Dead the dask cluster
#client.close()

In [38]:
# General purpose
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
from scipy.spatial.distance import chebyshev
import pandas as pd

# Seed
seed = 15051991 # My birthday because I love myself <3

# Joblib backend
from joblib import parallel_backend

# Dask

from dask_ml.model_selection import GridSearchCV
import dask.distributed
from dask.distributed import Client
from dask_ml.cluster import KMeans, PartialMiniBatchKMeans, SpectralClustering
#client = Client('scheduler-address:8786') # dask web interface client
client = Client()

# Sklearn
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift
from sklearn.model_selection import train_test_split

# Fuzzy clustering
import skfuzzy as fuzz


/Users/ignacio/anaconda3/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [30]:
df = pd.read_table('spiral.txt', names=['x','y'])
print('Data shape: {}'.format(df.shape))
df.head()

Data shape: (312, 2)


,x,y
0,31.95,7.95
1,31.15,7.30
2,30.45,6.65
3,29.70,6.00
4,28.90,5.55


In [34]:
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = seed)
print('Shape train: {}'.format(df_train.shape))
print('Shape test: {}'.format(df_test.shape))

Shape train: (218, 2)
Shape test: (94, 2)


In [47]:

mini_pipe = Pipeline(steps=[('kmeans', KMeans)])

pipe = Pipeline(steps=[
                        ('kmeans', KMeans),
                        ('partial_kmeans', PartialMiniBatchKMeans)
                      ]
               )

#Parameters of pipelines can be set using ‘__’ separated parameter names:
grid = dict(
            # K-Means
            kmeans__n_clusters=np.linspace(2, 200, 50),
            partial_kmeans__n_clusters=np.linspace(2,200,50),
            # Agglomeraive
            agglomerative_single_link__n_clusters=np.linspace(2, 200, 50),
            agglomerative_complete_link__n_clusters=np.linspace(2, 200, 50),
            # DBSCAN
            dbscan__eps=np.logspace(1, 100, 50),
            dbscan__min_samples=np.linspace(1,np.sqrt(df.shape[0]), 20),
            dbscan__metric=['manhattan','euclidean', 'cosine', chebyshev],
            # MeanShift
            meanshift__bandwidth=range(1,10,1),
            meanshift__cluster_all=[True,False],
                # True: Complete clustering
                # False: Partial clustering
                # Spectral Clustering
            spectral__n_clusters=np.linspace(2, 200, 50),
            )

# from sklearn.grid_search import GridSearchCV


estimator = GridSearchCV(mini_pipe, grid)


with parallel_backend('dask.distributed', scheduler_host='scheduler-address:8786'):
    estimator.fit(X, y)
    # your now-cluster-ified sklearn code here



KeyError: 'dask.distributed'

In [70]:
mini_pipe = Pipeline(steps=[('kmeans', KMeans)])
mini_grid = dict(kmeans__n_clusters=np.linspace(2,200,50))

grid = dict(
            # K-Means
            kmeans__n_clusters=np.linspace(2, 200, 50),
            kmeans__n_jobs=[5],
            partial_kmeans__n_clusters=np.linspace(2,200,50),
            # Agglomeraive
            agglomerative_single_link__n_clusters=np.linspace(2, 200, 50),
            agglomerative_complete_link__n_clusters=np.linspace(2, 200, 50),
            # DBSCAN
            dbscan__eps=np.logspace(1, 100, 50),
            dbscan__min_samples=np.linspace(1,np.sqrt(df.shape[0]), 20),
            dbscan__metric=['manhattan','euclidean', 'cosine', chebyshev],
            # MeanShift
            meanshift__bandwidth=range(1,10,1),
            meanshift__cluster_all=[True,False],
                # True: Complete clustering
                # False: Partial clustering
                # Spectral Clustering
            spectral__n_clusters=np.linspace(2, 200, 50),
)

# from sklearn.grid_search import GridSearchCV


estimator = GridSearchCV(KMeans(), mini_grid)

#with parallel_backend('dask.distributed', scheduler_host='scheduler-address:8786'):
estimator.fit(df_train, n_jobs = 5)

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [7]:
import numpy as np
np.linspace()

Object `np.linpace` not found.


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)